In [1]:
!pip install faiss-cpu openai sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found

In [3]:
import numpy as np
import pickle
import faiss
from sentence_transformers import SentenceTransformer

# Load your fine-tuned model
model = SentenceTransformer("/kaggle/input/output-from2020/kaggle/working/fine-tuned-legal-embedding-2020")

# Load FAISS index
index = faiss.read_index("/kaggle/input/output-from2020/kaggle/working/faiss_index.idx")

# Load metadata
with open("/kaggle/input/output-from2020/kaggle/working/faiss_metadata.pkl", 'rb') as f:
    metadata = pickle.load(f)

# Load embeddings
embeddings = np.load("/kaggle/input/output-from2020/kaggle/working/embeddings.npy")

print(f"✅ Model loaded successfully")
print(f"✅ FAISS index loaded: {index.ntotal} embeddings")
print(f"✅ Metadata loaded: {len(metadata)} documents")
print(f"✅ Embeddings shape: {embeddings.shape}")


2025-05-24 23:02:58.393936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748127778.602217      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748127778.662257      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ Model loaded successfully
✅ FAISS index loaded: 106745 embeddings
✅ Metadata loaded: 106745 documents
✅ Embeddings shape: (106745, 384)


In [19]:
import requests
import json

# Set your OpenRouter API key
OPENROUTER_API_KEY = ""  
def call_deepseek_v3(messages, temperature=0.7, max_tokens=1000):
    """Call DeepSeek V3 via OpenRouter API"""
    
    url = "https://openrouter.ai/api/v1/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://kaggle.com",  
        "X-Title": "Legal RAG Assistant",    
    }
    
    data = {
        "model": "deepseek/deepseek-chat-v3-0324:free", 
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    
    try:
        response = requests.post(url, headers=headers, data=json.dumps(data))
        response.raise_for_status()  
        
        result = response.json()
        return result['choices'][0]['message']['content']
        
    except requests.exceptions.RequestException as e:
        return f"API Request Error: {str(e)}"
    except KeyError as e:
        return f"Response Parse Error: {str(e)}"
    except Exception as e:
        return f"Unexpected Error: {str(e)}"


In [30]:
def translate_to_french(text):
    """Translate query to French for consistent embedding search"""
    messages = [
        {
            "role": "system",
            "content": "You are a translator. Translate the given text to French. ONLY return the translation, nothing else!"
        },
        {
            "role": "user", 
            "content": f"Translate this to French: {text}"
        }
    ]
    return call_deepseek_v3(messages, temperature=0.1, max_tokens=200)

def advanced_rag_query(query, top_k=5):
    """Advanced RAG with language detection and strict context-only responses"""
    
    # Store original query for language detection
    original_query = query
    
    # Translate query to French for embedding consistency
    french_query = translate_to_french(query)
    print(french_query)
    # Retrieve documents using French query
    query_embedding = model.encode([french_query])
    query_embedding = np.array(query_embedding).astype('float32')
    scores, indices = index.search(query_embedding, top_k)
    
    # Format context with document numbering
    context_parts = []
    for i, (score, idx) in enumerate(zip(scores[0], indices[0]), 1):
        if idx != -1:
            doc_text = metadata[idx]
            context_parts.append(f"Document {i}:\n{doc_text}")
    
    context = "\n\n".join(context_parts)
    # Language-adaptive prompt template
    prompt = f"""You are a legal assistant expert. Answer the user's question using ONLY the information provided in the documents below and with the same LANGUAGE as the user's question. 

IMPORTANT INSTRUCTIONS:
- Respond in the SAME LANGUAGE as this {original_query} question's language
- Use ONLY information from the provided documents
- DO NOT use your general knowledge or training data, if the answer is not mentioned in the CONTEXT DOCUMENTS say that i do not have enogh inforamation
- Reference specific documents in the end of the answer with a title is Sources:

CONTEXT DOCUMENTS provided:
{context}

USER QUESTION: {original_query}

CONSTRAINTS:
- Answer language: Same as the question language
- Information source: Only the provided documents above

ANSWER:"""

    messages = [
        {
            "role": "system", 
            "content": "You are a legal assistant. You must respond in the same language as the user's question and use ONLY the information from the provided context documents. Never use your general knowledge."
        },
        {
            "role": "user", 
            "content": prompt
        }
    ]
    
    return call_deepseek_v3(messages, temperature=0.2, max_tokens=1500)


In [21]:
# Test advanced RAG
print("\n🚀 Testing Advanced RAG...")
advanced_answer = advanced_rag_query("Quels sont les droits des travailleurs ?")
if not advanced_answer.startswith('Error'):
    print(f"Advanced Answer:\n{advanced_answer}")
else:
    print(f"Error: {advanced_answer}")


🚀 Testing Advanced RAG...
Quels sont les droits des travailleurs ?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Advanced Answer:
Les droits des travailleurs, tels que mentionnés dans les documents fournis, incluent notamment :

1. **Droit syndical** :  
   - Les travailleurs ont le droit de constituer librement des organisations syndicales de base (Document 2, art. 19).  
   - Ils peuvent adhérer librement et volontairement à une organisation syndicale de leur choix (Document 4, art. 6).  
   - L'employeur doit traiter à égalité les organisations syndicales et s'abstenir de toute ingérence (Document 4, art. 7).  

2. **Droit de grève** :  
   - Le droit de grève est reconnu, mais son exercice est encadré par la loi (Document 5, art. 70).  
   - Certaines limitations existent, notamment pour les services publics essentiels où un service minimum doit être assuré (Document 1, art. 62-64).  

3. **Protections fondamentales** :  
   - Droit à la protection, à la sécurité et à l'hygiène dans le travail (Document 5, art. 66).  
   - Droit au repos et à la sécurité sociale (Document 5, art. 66).  
   - 

In [31]:
# Test advanced RAG
print("\n🚀 Testing Advanced RAG...")
advanced_answer = advanced_rag_query("ما هي المراحل التي يمر بها القانون قبل أن يُنشر في الجريدة الرسمية؟")
if not advanced_answer.startswith('Error'):
    print(f"Advanced Answer:\n{advanced_answer}")
else:
    print(f"Error: {advanced_answer}")


🚀 Testing Advanced RAG...
Quelles sont les étapes que traverse une loi avant d'être publiée au Journal officiel ?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Advanced Answer:
لا تتوفر معلومات كافية في الوثائق المقدمة حول المراحل التي يمر بها القانون قبل نشره في الجريدة الرسمية.  

Sources: الوثائق 1-5 المقدمة


In [32]:
# Test advanced RAG
print("\n🚀 Testing Advanced RAG...")
advanced_answer = advanced_rag_query("Quels est le résultat de 5+5 ?")
if not advanced_answer.startswith('Error'):
    print(f"Advanced Answer:\n{advanced_answer}")
else:
    print(f"Error: {advanced_answer}")


🚀 Testing Advanced RAG...
rasta


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Advanced Answer:
Je ne dispose pas d'assez d'informations pour répondre à cette question.

Sources: Documents 1, 2, 3, 4, 5
